# Exercise 4, answers

In [26]:
def f_constrained(x):
    return x[0]**2+x[1]**2+x[0]+2*x[1], [], [x[0]+x[1]-1]

In [27]:
def alpha(x,f):
    (_,ieq,eq) = f(x)
    return sum([min([0,ieq_j])**2 for ieq_j in ieq])\
+sum([eq_k**2 for eq_k in eq])

def penalized_function(x,f,r):
    return f(x)[0] + r*alpha(x,f)

#### Let us solve the penalized problem with penalty term growin in a loop

In [43]:
import numpy as np
from scipy.optimize import minimize
r = 1
x_old = np.array([float('inf')]*2)
x_new = [0,0]
while np.linalg.norm(x_new-x_old)>0.0001:
    res = minimize(lambda x:penalized_function(x,f_constrained,r),
               [0,0],method='Nelder-Mead')
    x_old = x_new
    x_new = np.array(res.x)
    r = r+1
print(x_new, r)

[0.74340989 0.24336301] 95


## Problem 2

In [44]:
def f_constrained_approx(x,epsilon):
    return x[0]**2+x[1]**2+x[0]+2*x[1], [x[0]+x[1]-1+epsilon,\
                                         epsilon-(x[0]+x[1]-1)], []

In [45]:
def beta(x,f):
    _,ieq,_ = f(x)
    try:
        value=sum([1/max([0,ieq_j]) for ieq_j in ieq])
    except ZeroDivisionError:
        value = float("inf")
    return value
def function_with_barrier(x,f,r):
    return f(x)[0]+r*beta(x,f)

In [49]:
import numpy as np
import ad
from scipy.optimize import minimize
r = 1.0
epsilon = .1
x_old = np.array([float('inf')]*2)
x_new = [1,0]
while np.linalg.norm(x_new-x_old)>0.0001:
    g = lambda x: function_with_barrier(x,\
               lambda y: f_constrained_approx(y,epsilon),r)
    res = minimize(g,x_new,method='Newton-CG',jac=ad.gh(g)[0],\
                   hess=ad.gh(g)[1])
    x_old = x_new
    x_new = res.x
    r=r/2
    epsilon = epsilon/2
    print(x_new, epsilon, r)

[0.7496876 0.2496876] 0.05 0.5
[0.74992188 0.24992188] 0.025 0.25
[0.74998047 0.24998047] 0.0125 0.125


## Problem 3

In [50]:
import numpy as np
def project_vector(A,vector):
    #convert A into a matrix
    A_matrix = np.matrix(A)
    #construct the "first row" of the matrix [[I,A^T],[A,0]]
    left_matrix_first_row = np.concatenate((np.identity(len(vector)),A_matrix.transpose()), axis=1)
    #construct the "second row" of the matrix
    left_matrix_second_row = np.concatenate((A_matrix,np.matrix(np.zeros([len(A),len(A)]))), axis=1)
    #combine the whole matrix by combining the rows
    left_matrix = np.concatenate((left_matrix_first_row,left_matrix_second_row),axis = 0)
    #Solve the system of linear equalities from the previous page
    return np.linalg.solve(left_matrix, \
                           np.concatenate((np.matrix(vector).transpose(),\
                                           np.zeros([len(A),1])),axis=0))[:len(vector)]         

In [51]:
import numpy as np
import ad
def projected_gradient_method(f,A,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    while abs(f_old-f_new)>precision:
        f_old = f_new
        gradient = ad.gh(f)[0](x)
        grad_proj = project_vector(A,[-i for i in gradient])#The only changes to steepest..
        grad_proj = np.array(grad_proj.transpose())[0] #... descent are here!
        x = x+grad_proj*step
        f_new = f(x)
        steps.append(list(x))
    return x,f_new,steps

In [58]:
projected_gradient_method(lambda x:f_constrained(x)[0],[[1,1]],[1,0]\
                          ,.2,0.000001)

(array([0.75032652, 0.24967348]),
 1.8750002132271604,
 [[0.9, 0.1],
  [0.8400000000000001, 0.15999999999999998],
  [0.804, 0.196],
  [0.7824, 0.21760000000000002],
  [0.76944, 0.23056000000000001],
  [0.761664, 0.23833599999999996],
  [0.7569984, 0.2430016],
  [0.7541990399999999, 0.24580096000000004],
  [0.7525194239999999, 0.24748057600000006],
  [0.7515116544, 0.24848834560000005],
  [0.75090699264, 0.24909300736000006],
  [0.750544195584, 0.24945580441600007],
  [0.7503265173504, 0.24967348264960007]])

## Problem 4

Need to show that there exists unique Lagrance multiplier vectors $\lambda^* = (\lambda^*_1,\ldots,\lambda_J^*)$ and $\mu^*=(\mu_1^*,\ldots,\mu_K^*)$ such that
$$
\begin{align}
&\nabla_xL(x,\lambda,\mu) = 0\\
&\mu_j^*\geq0,\text{ for all }j=1,\ldots,J\\
&\mu_j^*g_j(x)=0,\text{for all }j=1,\ldots,J,
\end{align}
$$
where $$L(x,\lambda,\mu) = f(x)- \sum_{k=1}^K\mu_kg_k(x) -\sum_{j=1}^J\lambda_jh_j(x)$$

Now,  $f(x) = x_1^2+x_2^2+x_1+2x_2$, $g(x) = 0$ and $h(x)=x_1+x_2-1$.

Thus, stability rule becomes $$
\left\{
\begin{align}
2x_1+1-\lambda = 0\\
2x_2+2-\lambda=0.
\end{align}
\right.
$$


We do not have a complementary rule, since we do not have inequality constraints!

In [59]:
def check_KKT_eqc(x,tol):
    l = 2*x[0]+1
    if abs(2*x[1]+2-l)<=tol:
        print(abs(2*x[1]+2-l))
        return True
    return False

In [61]:
check_KKT_eqc([0.74998093,0.24998093],0.000001)
check_KKT_eqc([0.75,0.25],0.000001)

0.0
0.0


True